In [3]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import convolve2d
from scipy.fft import fft2
from scipy.ndimage import gaussian_filter
from skimage.filters import unsharp_mask
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import convolve2d
import cv2
import numpy as np


In [17]:
color_image = cv2.imread('imgNocturna.png').astype(np.float64)

# Parameters
cutoff = 10 #frecuencia a la cual se produce la atenuación
order = 2 #atenuación sea más o menos pronunciada
Yh = 0.0999 
Yl = 1.01

homomorphic_image = np.zeros_like(color_image)

for i in range(color_image.shape[2]):
    color_channel = color_image[:, :, i]

    # Add a small offset to avoid division by zero
    epsilon = 1e-8
    color_channel_offset = color_channel + epsilon

    # Calculate the logarithm
    color_channel_logued = np.log(1 + color_channel_offset)

    # DFT
    color_channel_dft = np.fft.fft2(color_channel_logued)

    # Filter
    x, y = color_channel.shape
    A = np.zeros((x, y))
    H = np.zeros((x, y))
    d = cutoff
    n = order

    for j in range(x):
        for k in range(y):
            A[j, k] = np.sqrt((j - x / 2) ** 2 + (k - y / 2) ** 2)
            if A[j, k] == 0:
                H[j, k] = 0
            else:
                H[j, k] = 1 / (1 + ((d / A[j, k]) ** (2 * n)))

    H = ((Yl - Yh) * H) + Yh
    H = 1 - H

    color_channel_f = H * color_channel_dft

    # Inverse DFT
    color_channel_n = np.abs(np.fft.ifft2(color_channel_f))

    # Inverse log
    color_channel_final = np.exp(color_channel_n)

    # Update the original color channel with the processed values
    homomorphic_image[:, :, i] = color_channel_final

# Normalize and convert to uint8
homomorphic_image = ((homomorphic_image - np.min(homomorphic_image)) / (np.max(homomorphic_image) - np.min(homomorphic_image)) * 255).astype(np.uint8)

#Normalizar y suavizar:

# Save the result
cv2.imwrite('homomorphicColor1.png', homomorphic_image) #Homographic es del TP3

True

In [14]:
# Cargar la imagen en color
image = cv2.imread('IMG_6932.png')

# Convertir la imagen a escala de grises
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Oscurecer la imagen en escala de grises
factor_oscurecimiento = 0.4
darkened_gray_image = (gray_image * factor_oscurecimiento).astype(np.uint8)

# Valor a usar para el high-boost
A = 1.5

mask = np.array([[-1, -1, -1],
                [-1, A + 8, -1],
                [-1, -1, -1]], dtype=np.float32)

# Aplicar el filtro Laplaciano
hb_laplacian = cv2.filter2D(darkened_gray_image, -1, mask)

# Sumar el resultado al canal original
hb_image = cv2.addWeighted(image, 1.0, cv2.cvtColor(hb_laplacian, cv2.COLOR_GRAY2BGR), 1.0, 0)

# Para limitar los valores máximos de brillo (>255 ya satura)
hb_image = np.clip(hb_image, 0, 255).astype(np.uint8)

# Guardar las imágenes
cv2.imwrite('results/original_image_2.png', image)
cv2.imwrite('results/high_boosted_image_2.png', hb_image) #HIGH BOOST TP3

True

In [32]:
# Aplicar suavizado 

# Cargar la imagen en color
image = cv2.imread('homomorphicColor2.png')
homomorphic_image_smoothed =  cv2.blur(image, (15, 15))

# Guardar el resultado
cv2.imwrite('homomorphicColor2_smoothed.png', homomorphic_image_smoothed)

True

In [35]:
#Lapplaciano tp3 TARDA 20 MINUTOS
image = cv2.imread('homomorphicColor2.png')


# Definir las máscaras
normal_mask = np.array([[1, 1, 1],
                        [1, -8, 1],
                        [1, 1, 1]], dtype=np.float32)

borders_mask = np.array([[-1, -1, -1],
                         [-1, 9, -1],
                         [-1, -1, -1]], dtype=np.float32)

height, width, channels = image.shape

# Inicializar las matrices laplacianas por canal
normal_laplacian = np.zeros((height, width, channels), dtype=np.float32)
borders_laplacian = np.zeros((height, width, channels), dtype=np.float32)

# Aplicar las máscaras por canal
for i in range(1, height - 1):
    for j in range(1, width - 1):
        region = image[i-1:i+2, j-1:j+2, :]
        for channel in range(channels):
            normal_value = np.sum(region[:, :, channel] * normal_mask)
            normal_laplacian[i, j, channel] = normal_value
            borders_value = np.sum(region[:, :, channel] * borders_mask)
            borders_laplacian[i, j, channel] = borders_value

# Aplicar el clip y cambio de escala por canal
normal_laplacian = np.clip(normal_laplacian, 0, 255).astype(np.uint8)
borders_laplacian = np.clip(borders_laplacian, 0, 255).astype(np.uint8)

normal_laplacian_scaled_down = np.clip(normal_laplacian * 0.5, 0, 255).astype(np.uint8)
normal_laplacian_scaled_up = np.clip(normal_laplacian * 1.5, 0, 255).astype(np.uint8)

# Restar el laplaciano a la imagen original por canal
image_with_normal_mask = cv2.subtract(image, normal_laplacian)
image_with_normal_mask_scaled_down = cv2.subtract(image, normal_laplacian_scaled_down)
image_with_normal_mask_scaled_up = cv2.subtract(image, normal_laplacian_scaled_up)

In [ ]:
# Especifica las rutas de guardado para cada imagen
path_normal_laplacian = 'normal_laplacian.png'
path_borders_laplacian = 'borders_laplacian.png'
path_normal_laplacian_scaled_down = 'normal_laplacian_scaled_down.png'
path_normal_laplacian_scaled_up = 'normal_laplacian_scaled_up.png'
path_image_with_normal_mask = 'image_with_normal_mask.png'
path_image_with_normal_mask_scaled_down = 'image_with_normal_mask_scaled_down.png'
path_image_with_normal_mask_scaled_up = 'image_with_normal_mask_scaled_up.png'

# Guardar las imágenes
cv2.imwrite(path_normal_laplacian, normal_laplacian)
cv2.imwrite(path_borders_laplacian, borders_laplacian)
cv2.imwrite(path_normal_laplacian_scaled_down, normal_laplacian_scaled_down)
cv2.imwrite(path_normal_laplacian_scaled_up, normal_laplacian_scaled_up)
cv2.imwrite(path_image_with_normal_mask, image_with_normal_mask)
cv2.imwrite(path_image_with_normal_mask_scaled_down, image_with_normal_mask_scaled_down)
cv2.imwrite(path_image_with_normal_mask_scaled_up, image_with_normal_mask_scaled_up)

In [ ]:
#filtro unsharp tp2

In [ ]:
#corrección gamma tp color

In [ ]:
#interpolacion? 

In [ ]:
from PIL import Image, ImageEnhance

# Open the image
image = Image.open('high_boosted_image.png')

# Create an ImageEnhance object and adjust the contrast
# (factor > 1 increases contrast, factor < 1 decreases contrast)
factor = 0.5  # You can adjust this value
enhancer = ImageEnhance.Contrast(image)
adjusted_image = enhancer.enhance(factor)


# Save the adjusted image
adjusted_image.save(f'results/contrast{factor}.jpg')


In [ ]:
#HDR	